In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
import math
from scipy.optimize import minimize
from scipy.spatial import ConvexHull
from scipy.linalg import norm
from scipy.optimize import minimize_scalar

path= 'C:\\Users\\matteo posenato\\Documents\\optimization\\project\\banana.csv'
data = pd.read_csv(path, sep=',')

In [2]:
def regularize_data(data):
    data = data[data.At1 < 1]
    data = data[data.At1 > -1]
    data = data[data.At2 < 1]
    data = data[data.At2 > -1]
    data = data.reset_index(drop = True)
    return data
def canonical_basis_vector(size, index):
    vector = np.zeros(size)
    vector[index] = 1
    return vector
def create_d_t(d, A, w):
    m = len(d)
    delta = np.sum(d * np.log(d)) + np.log(m)
    result = np.dot(d.T, np.dot(A, w))
    result += (1/m) * delta
    return result

In [3]:
data_r = regularize_data(data)

In [4]:
def weak_learner(data, weights=None):
    X = data.iloc[:, :-1]
    y = data.Class
    model = DecisionTreeClassifier(max_depth=2)
    if weights is not None:
        model.fit(X, y, sample_weight=weights)
    else:
        model.fit(X,y)
    h = model.predict(X)
    #return our hypothesis space
    return h, X, y

In [5]:
h, X, y = weak_learner(data_r)

In [6]:
def create_A(h, X, y):
    m = len(h)  
    #create A
    A = np.zeros((m, m))
    for i in range(m):
        for j in range(m):
            value = y[i] * h[j]
            A[i][j] = value
    return A, m

In [7]:
A, m = create_A(h, X, y)

In [8]:
d0 = np.zeros(m)
for i in range(m):
    d0[i] = 1/m 

In [9]:
hj_1, X, y = weak_learner(data_r, weights=d0)
e_j = canonical_basis_vector(m, 1)
w = np.zeros(m)
w[1] = e_j[1]

In [12]:
#set the paratemeter T 
T = 5
#create all the variables that we need
E_t = []
E_t.insert(0, e_j)
lamb = []
for t in range(1, T):
    #create the distribution d_t
    t_n = t + 1
    result = minimize(create_d_t, x0=d0, args=(A, w))
    d0[t] = result.x[1]
    #obtain the hypothesis h_j+1 and e_j+1
    e_j_next = canonical_basis_vector(m, t_n)
    h_j_nect = h[t_n]
    E_t.insert(t_n, e_j_next)
    #compute the parwise weight
    w_1= []
    my_f = lambda x, A, e_j: np.dot(d0.T, np.dot(A, x))
    e_away = minimize(my_f, x0=e_j, args=(A, d0)).x
    def compute_alpha(w, e):
        alpha.append(w[np.nonzero(e)])
        return alpha
    alpha = []
    for e in E_t:
        alpha = compute_alpha(w, e)
    lamb_max = compute_alpha(e_away, w)
    def my_f(lamb, w, e_j_next, e_away, d0, A):
        r = w + lamb * (e_j_next - e_away)
        my_f_value = np.dot(d0.T, -A.dot(r))
        return my_f_value
    lamb = minimize_scalar(my_f, bounds=(0, lamb_max[t]), args=(w, e_j_next, e_away, d0, A)).x
    w_1.insert(0, w + lamb* (e_j_next - e_away))
    #compute the secondary weight
    w_2 = []
    my_f = lambda x, A, w: np.dot(d0.T, np.dot(A, x))
    result = minimize(my_f, x0=d0, args=(A, w))
    w_2.append(result.x)
    #update the weight
    w_f = np.array([w_1[0], w_2[0]])
    def f(w):
        return np.dot(d0.T, -np.dot(A, w))
    def f_1(x, w):
        if x == 0:
            return f(w[0])
        elif x == 1:
            return f(w[1])

    # Trova l'argomento che minimizza la funzione per il primo vettore
    result_1 = minimize_scalar(f_1, args=(w_f,), bounds=(0, 0), method='bounded')
    arg_min_1 = result_1.x

    # Trova l'argomento che minimizza la funzione per il secondo vettore
    result_2 = minimize_scalar(f_1, args=(w_f,), bounds=(1, 1), method='bounded')
    arg_min_2 = result_2.x
    # Confronta i risultati per determinare quale valore minimizza di più la funzione
    if arg_min_1 < arg_min_2:
        # Il primo vettore minimizza di più la funzione
        print("Il primo vettore minimizza di più la funzione.")
        print("argmin1: {}".format(arg_min_1))
        w[t_n] = arg_min_1
    elif arg_min_1 > arg_min_2:
        # Il secondo vettore minimizza di più la funzione
        print("Il secondo vettore minimizza di più la funzione.")
        print("argmin2: {}".format(arg_min_2))
        w[t_n] = arg_min_2
    else:
        # Entrambi i valori sono uguali
        print("Entrambi i valori minimizzano la funzione allo stesso modo.")

C:\Users\matteo posenato\AppData\Local\Temp\ipykernel_10232\1915115368.py:14: RuntimeWarning: invalid value encountered in log
  delta = np.sum(d * np.log(d)) + np.log(m)


Il primo vettore minimizza di più la funzione.
argmin1: 0.0
Il primo vettore minimizza di più la funzione.
argmin1: 0.0
Il primo vettore minimizza di più la funzione.
argmin1: 0.0
Il primo vettore minimizza di più la funzione.
argmin1: 0.0


In [1]:
def pairwise_FW(A, d, E_t):
    best_e = None
    E_t = E_t_plus_one[:-1]
    for e in E_t:
        value = expression_f(A, d, e)
        if value < best_e:
            best_e = value
    return best_e
def expression_f(A, d, e):
    return np.dot(d.T, np.dot(A, e))
def step_size(s):
    value_0 = (ej_2 - e_away)
    value_1 = s(value_0)
    value_2 = w + value_1
    value_3 = np.dot(-A, value_2)
    value = value_3 + (1/N)* delta(result)
e_away = pairwise_FW(A, result, E_T_plus_one)
a = []
for i in range(len(e_away)):
    if e_away[i] == w[i]:
        a[i] = 1
    else:
        a[i] = 0
#set lamb_t_max
lamb_t_max = a
#calculate lamb_t
best_lamb = None 
for s in range(0:lamb_t_max):
    value = step_size(s)
    if value < best_lamb:
        best_lamb = value
w_1 = w + best_lamb * (ej_2 - e_away)
